<a href="https://colab.research.google.com/github/holictoweb/python_finance_analysis/blob/master/backtrader/toone_backtrading_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## bactrader


In [107]:
import pandas as pd
import matplotlib as mat
import yfinance as yf
import backtrader as bt
from datetime import datetime, timedelta, timezone
 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType
from pyspark.sql.types import *
from pyspark.sql.functions import lit
 
spark = SparkSession.builder.appName("toone_trader_01").getOrCreate()

## 0. target code list 

### data list 에 대한 부분은 parquet data로 처리 예정 
- 데이터를 가져 오기 위한 대상 타겟 수집 

In [109]:
target_list = ["005930.KS", '068270.KS', '035420.KS', '064260.KQ']
 
#target_list = "SPY AAPL MSFT"
target_str = ' '.join(target_list)

## 1. ingest data from yfinance

### data to pandas dataframe

- min 단위의 데이터를 6일치 (5영업일) 데이터를 가져 옴. 

In [125]:

#7일 전까지만 지정 가능
targate_date = datetime.now(timezone.utc) + timedelta(hours=-9) - timedelta(days=6)
print(targate_date)
sdate = targate_date

#sdate = datetime(2021, 4,20)
#edate = datetime.(2021, 3,31)

pdf_min = yf.download(  # or pdr.get_data_yahoo(...
  # tickers list or string as well
  #tickers = "SPY AAPL MSFT",
  tickers = target_str,

  # use "period" instead of start/end
  # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
  # (optional, default is '1mo')
  period = "ytd",

  # fetch data by interval (including intraday if period < 60 days)
  # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
  # (optional, default is '1d')
  interval = "1m",

  # group by ticker (to access via data['SPY'])
  # (optional, default is 'column')
  group_by = 'ticker',

  # adjust all OHLC automatically
  # (optional, default is False)
  auto_adjust = True,

  # download pre/post regular market hours data
  # (optional, default is False)
  prepost = True,

  # use threads for mass downloading? (True/False/Integer)
  # (optional, default is True)
  threads = True,

  # proxy URL scheme use use when downloading?
  # (optional, default is None)
  proxy = None,
  start=sdate 
  #end=edate
  )

2021-04-22 01:31:53.626563+00:00
[*********************100%***********************]  4 of 4 completed


In [133]:
# test check data 

#pdf_min['005930.KS'].reset_index().assign( Code = '005930.KS')
 
#pdf_min.reset_index()

pdf_min


035420.KS            ... 068270.KS        
                               Open      High  ...     Close  Volume
Datetime                                       ...                  
2021-04-22 10:32:00+09:00  382500.0  382500.0  ...  291000.0     0.0
2021-04-22 10:33:00+09:00  382000.0  382500.0  ...  291500.0  1513.0
2021-04-22 10:34:00+09:00  382000.0  382500.0  ...  291500.0   754.0
2021-04-22 10:35:00+09:00  382000.0  382500.0  ...  291000.0  1266.0
2021-04-22 10:36:00+09:00  382000.0  382500.0  ...  291500.0   892.0
...                             ...       ...  ...       ...     ...
2021-04-28 15:16:00+09:00  374000.0  374000.0  ...  271500.0  3039.0
2021-04-28 15:17:00+09:00  373500.0  374000.0  ...  272000.0  4696.0
2021-04-28 15:18:00+09:00  373500.0  374000.0  ...  272000.0  1599.0
2021-04-28 15:19:00+09:00  374000.0  374000.0  ...  271500.0  3506.0
2021-04-28 15:20:00+09:00  374000.0  374000.0  ...       NaN     NaN

[1813 rows x 20 columns]

2. save to spark lakehouse

## 3. spark data

- 데이터 수집을 위한 dataframe 생성 

- reset_index()  
[데이터사이언스 스쿨 ](https://datascienceschool.net/01%20python/04.05%20%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%94%84%EB%A0%88%EC%9E%84%20%EC%9D%B8%EB%8D%B1%EC%8A%A4%20%EC%A1%B0%EC%9E%91.html)

In [112]:
stock_min_schema = StructType([
  StructField('Datetime', TimestampType(), True),
  StructField('Open', DoubleType(), True),
  StructField('High', DoubleType(), True),
  StructField('Low', DoubleType(), True),
  StructField('Close', DoubleType(), True),
  StructField('Volume', DoubleType(), True),
  StructField('Code', StringType(), True),
  ])

emptyRDD = spark.sparkContext.emptyRDD()

df = spark.createDataFrame(emptyRDD,stock_min_schema)
df.printSchema()



root
 |-- Datetime: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- Code: string (nullable = true)



### lakewarhose 생성
- spark database 생성
- df 에서 google drive 상에 데이터 저장 -  parquet partition by code 

- parquet data를 기반으로 spark table 생성 

In [132]:
#ticker_list = ["005930.KS", '068270.KS', '035420.KS', '064260.KQ']
table_path = '/content/drive/MyDrive/data-warehouse/stock_min/'
table_name = 'stock_min'

# db 생성 
spark.sql("create database IF NOT EXISTS stocklab")
# table 삭제 
spark.sql("drop table if exists stocklab."+ table_name)
# table 위치 지정 



for ticker in target_list:  
  
  code = ticker.split('.')[0]
  pdf_extract = pdf_min[ticker].assign( Code = code )
  
  df_append = spark.createDataFrame( pdf_extract.reset_index(), stock_min_schema  ) 
  
  #df_append = df_append.withColumn("Code", lit(code))
  #df = df.withColumnRenamed("Adj Close", "AdjClose").withColumn("Code", lit(code))

  df = df.union(df_append)

# 추가된 모든 데이터를 다시 추가... 
df.write.format('parquet').mode("overwrite").partitionBy("Code").save(table_path )

create_table_sql = 'create table if not exists stocklab.'+ table_name+ ' using org.apache.spark.sql.parquet options (path "'+ table_path +'")'
#print(create_table_sql)
spark.sql(create_table_sql)

# 데이터 변경 분에 대한 refresh 작업 진행 
spark.sql('refresh table stocklab.' + table_name) #spark.catalog.refreshTable("my_table") database 지정 방법은??

spark.sql('describe table extended stocklab.' + table_name).show(truncate=False)


+----------------------------+----------------------------------------------------+-------+
|col_name                    |data_type                                           |comment|
+----------------------------+----------------------------------------------------+-------+
|Datetime                    |timestamp                                           |null   |
|Open                        |double                                              |null   |
|High                        |double                                              |null   |
|Low                         |double                                              |null   |
|Close                       |double                                              |null   |
|Volume                      |double                                              |null   |
|Code                        |int                                                 |null   |
|# Partition Information     |                                                  

In [122]:
spark.sql("use stocklab ").show()
spark.sql("show tables ").show()
spark.sql("select Code from stocklab.stock_min group by Code").show()
#df_min.show()


#df_target=df_min.groupBy('Code')
#df_target.show()

++
||
++
++

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|stocklab|stock_min|      false|
+--------+---------+-----------+

+-----+
| Code|
+-----+
| 5930|
|64260|
|35420|
|68270|
+-----+



 ## Class strategy 생성

https://www.programcreek.com/python/example/124327/backtrader.Strategy

- 

In [ ]:
class Strat1_2BD_5BH(bt.Strategy):   
    params = (
        ('exitbars',10), # Tuple of tuples containing any variable settings required by the strategy.
    )
    
    def __init__(self):
        self.dataclose= self.datas[0].close    # Keep a reference to the "close" line in the data[0] dataseries
        self.order = None # Property to keep track of pending orders.  There are no orders when the strategy is initialized.
        self.buyprice = None
        self.buycomm = None
    
    
    def log(self, txt, dt=None):
        # Logging function for the strategy.  'txt' is the statement and 'dt' can be used to specify a specific datetime
        dt = dt or self.datas[0].datetime.date(0)
        print('{0},{1}'.format(dt.isoformat(),txt))
    
    def notify_order(self, order):
        # 1. If order is submitted/accepted, do nothing 
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 2. If order is buy/sell executed, report price executed
        if order.status in [order.Completed]: 
            if order.isbuy():
                self.log('BUY EXECUTED, Price: {0:8.2f}, Cost: {1:8.2f}, Comm: {2:8.2f}'.format(
                    order.executed.price,
                    order.executed.value,
                    order.executed.comm))
                
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log('SELL EXECUTED, {0:8.2f}, Cost: {1:8.2f}, Comm{2:8.2f}'.format(
                    order.executed.price, 
                    order.executed.value,
                    order.executed.comm))
            
            self.bar_executed = len(self) #when was trade executed
        # 3. If order is canceled/margin/rejected, report order canceled
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
            
        self.order = None
    
    def notify_trade(self,trade):
        if not trade.isclosed:
            return
        
        self.log('OPERATION PROFIT, GROSS {0:8.2f}, NET {1:8.2f}'.format(
            trade.pnl, trade.pnlcomm))
    
    def next(self):
        # Log the closing prices of the series from the reference
        self.log('Close, {0:8.2f}'.format(self.dataclose[0]))
 
        if self.order: # check if order is pending, if so, then break out
            return
                
        # since there is no order pending, are we in the market?    
        if not self.position: # not in the market
            if self.dataclose[0] < self.dataclose[-1]:
                if self.dataclose[-1] < self.dataclose[-2]:
                    self.log('BUY CREATE {0:8.2f}'.format(self.dataclose[0]))
                    self.order = self.buy()           
        else: # in the market
            if len(self) >= (self.bar_executed+self.params.exitbars):
                self.log('SELL CREATE, {0:8.2f}'.format(self.dataclose[0]))
                self.order = self.sell()

In [ ]:
cerebro = bt.Cerebro(stdstats = False)

cerebro.adddata(data)
cerebro.broker.setcash(1000.0)
cerebro.addstrategy(SmaStrategy)
cerebro.addobserver(bt.observers.BuySell)
cerebro.addobserver(bt.observers.Value)

In [1]:
# 기본 환경 구성 
 
!pip install pyspark
!pip install yfinance
!pip install backtrader
 
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/data-warehouse/

     |████████████████████████████████| 212.3MB 65kB/s 
     |████████████████████████████████| 204kB 45.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=1a3783ca46edc21ffdb6f54de64b540d679626176f49fe4141588591db96acca
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
     |████████████████████████████████| 5.5MB 14.3MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=23381b345336762ee4d06ac8d6f8d53809e70475b98053d6e60d79654453a2fc
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 419kB 12.0MB/s 
Mounted at /content/drive
stock_day  stocklab_db	stock_min
